In [386]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from scipy.stats import norm
from arch import arch_model
from scipy.stats import binom
import scipy.stats as stats
from numpy.random import default_rng
from scipy.stats import kstest
from numpy.random import Generator, PCG64, SeedSequence
import statsmodels.formula.api as smf
from statsmodels.regression.linear_model import RegressionResults
from scipy.stats import t
import statsmodels.api as sm
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import grangercausalitytests

### QUESTION 1 :
Un article récent de Welch (2022) révèle que les traders RH détiennent en fait un portefeuille
diversifié qui a, au moins depuis 2018, surperformé le marché S&P 500. Nous commencerons
notre analyse en nous faisant une idée de la performance des traders RH au cours des dernières
annéees. Téléchargez les données RH à partir du site web du cours et écrivez une fonction qui
utilisera les données sur les avoirs des utilisateurs RH pour déterminer ce que les investisseurs
RH ont détenu en moyenne au cours des dernières années.

In [387]:
df_crsp = pd.read_csv('RH_data.csv', parse_dates=['date'])
df_crsp['date'] = pd.to_datetime(df_crsp['date'])
df_crsp


,date,users_holding,tic
0,2019-01-01,175176,AAL
1,2019-01-01,55824,AAOI
2,2019-01-01,5204232,AAPL
3,2019-01-01,125592,AAXN
4,2019-01-01,42504,ABB
...,...,...,...
436581,2020-08-13,4396732,ZNGA
436582,2020-08-13,1113509,ZOM
436583,2020-08-13,147683,ZS
436584,2020-08-13,163777,ZSAN


In [388]:
# Import data from S&P 500

df_SP500 = pd.read_csv('S_P 500 Historical Data.csv', parse_dates=['Date'])

# Remove thousand separator from values
df_SP500['Price'] = df_SP500['Price'].str.replace(',', '')
df_SP500['Open'] = df_SP500['Open'].str.replace(',', '')
df_SP500['High'] = df_SP500['High'].str.replace(',', '')
df_SP500['Low'] = df_SP500['Low'].str.replace(',', '')
df_SP500['Change %'] = df_SP500['Change %'].str.replace('%', '')


# Change data type from object to float
df_SP500['Price'] = df_SP500['Price'].astype(float)
df_SP500['Open'] = df_SP500['Open'].astype(float)
df_SP500['High'] = df_SP500['High'].astype(float)
df_SP500['Low'] = df_SP500['Low'].astype(float)
df_SP500['Change %'] = df_SP500['Change %'].astype(float)

# Take only the date from 2019-01-01 to 2020-08-13 inclusively : 

start_date = '2019-01-01'
end_date = '2020-08-13'
df_SP500_selected = df_SP500.loc[(df_SP500['Date'] >= start_date) & (df_SP500['Date'] <= end_date)]

df_SP500_selected

,Date,Price,Open,High,Low,Vol.,Change %
313,2020-08-13,3373.43,3372.95,3387.24,3363.35,NaN,-0.20
314,2020-08-12,3380.35,3355.46,3387.89,3355.46,NaN,1.40
315,2020-08-11,3333.69,3370.34,3381.01,3326.44,NaN,-0.80
316,2020-08-10,3360.47,3356.04,3363.29,3335.44,NaN,0.27
317,2020-08-07,3351.28,3340.05,3352.54,3328.72,NaN,0.06
...,...,...,...,...,...,...,...
716,2019-01-08,2574.41,2568.11,2579.82,2547.56,NaN,0.97
717,2019-01-07,2549.69,2535.61,2566.16,2524.56,NaN,0.70
718,2019-01-04,2531.94,2474.33,2538.07,2474.33,NaN,3.43
719,2019-01-03,2447.89,2491.92,2493.14,2443.96,NaN,-2.48


In [389]:
df_stock_data = pd.read_csv('stock_data.csv')
df_stock_data['DlyCalDt'] = pd.to_datetime(df_stock_data['DlyCalDt'], format='%d%b%Y')
df_stock_data

,Ticker,DlyCalDt,DlyPrc,DlyRet,DlyVol,vwretd
0,ORCL,2019-01-02,45.2200,0.001550,14320441.0,0.001796
1,MSFT,2019-01-02,101.1200,-0.004430,35347045.0,0.001796
2,HON,2019-01-02,131.8300,-0.002195,2756347.0,0.001796
3,AKRX,2019-01-02,3.4800,0.026549,5343499.0,0.001796
4,DSS,2019-01-02,0.7981,0.089408,36364.0,0.001796
...,...,...,...,...,...,...
539762,OAS,2020-08-13,0.7400,-0.038836,11777116.0,-0.000593
539763,QEP,2020-08-13,1.5000,-0.025974,3638967.0,-0.000593
539764,SIX,2020-08-13,20.6000,0.040404,2293695.0,-0.000593
539765,BNO,2020-08-13,11.7000,-0.004255,382270.0,-0.000593


Dans cette étape, nous supposons que les avoirs de l'utilisateur sont les avoirs de l'investisseur représentatif pour cette action à un moment donné.

In [390]:
import pandas as pd

# Je commence par regrouper les données du fichier RH par "ticker" ou par actions : 
grouped_data = df_crsp.groupby("tic")

# Je calcule la somme totale des avoirs pour chacunes des actions à chaque jour
total_holdings = grouped_data["users_holding"].sum()

# Je calcule les poids en divisant chaque holding par la somme des holdings à chacun des jours.
for tic, group in grouped_data:
    df_crsp.loc[group.index, "weight"] = group["users_holding"] / total_holdings[tic]


Le code ci haut permet de calculer les poids pour chaque action en divisant chaque holding par la somme des holdings de chaque jour pour chaque actions. Les poids calculés sont ensuite stockés dans la colonne "weight" du DataFrame df_crsp.

In [391]:
df_crsp

,date,users_holding,tic,weight
0,2019-01-01,175176,AAL,0.000075
1,2019-01-01,55824,AAOI,0.004556
2,2019-01-01,5204232,AAPL,0.001268
3,2019-01-01,125592,AAXN,0.001583
4,2019-01-01,42504,ABB,0.003376
...,...,...,...,...
436581,2020-08-13,4396732,ZNGA,0.002339
436582,2020-08-13,1113509,ZOM,0.007685
436583,2020-08-13,147683,ZS,0.001845
436584,2020-08-13,163777,ZSAN,0.013922


In [398]:
merged_df = pd.merge(df_crsp, df_stock_data, left_on=['tic', 'date'], right_on=['Ticker', 'DlyCalDt'])

merged_df['portfolio_return'] = merged_df['weight'] * merged_df['DlyRet']

portfolio_returns = merged_df.groupby(['users_holding', 'date'])['portfolio_return'].sum()

df_portfolio_returns = portfolio_returns.to_frame().reset_index()
print(df_portfolio_returns)


        users_holding       date  portfolio_return
0               17010 2019-02-06          0.000097
1               17100 2019-02-06          0.000009
2               17110 2019-02-06         -0.000003
3               17120 2019-02-06          0.000003
4               17180 2019-02-06         -0.000009
...               ...        ...               ...
290187      112113479 2020-07-20         -0.000367
290188      112414463 2020-07-16         -0.000240
290189      126309687 2020-07-16          0.000358
290190      132757748 2020-07-17          0.000064
290191      147257338 2020-07-17         -0.000205

[290192 rows x 3 columns]


In [402]:
df_crsp = df_crsp.merge(df_portfolio_returns[['portfolio_return']], left_index=True, right_index=True)


In [403]:
df_crsp

,date,users_holding,tic,weight,portfolio_return
0,2019-01-01,175176,AAL,0.000075,0.000097
1,2019-01-01,55824,AAOI,0.004556,0.000009
2,2019-01-01,5204232,AAPL,0.001268,-0.000003
3,2019-01-01,125592,AAXN,0.001583,0.000003
4,2019-01-01,42504,ABB,0.003376,-0.000009
...,...,...,...,...,...
290187,2020-02-14,73423,UL,0.003877,-0.000367
290188,2020-02-14,86840,ULTA,0.001725,-0.000240
290189,2020-02-14,104323,UMC,0.002122,0.000358
290190,2020-02-14,64858,UNFI,0.002824,0.000064


In [400]:


# Compute basic statistics of portfolio returns
portfolio_mean = portfolio_returns['portfolio_return'].mean()
portfolio_std = portfolio_returns['portfolio_return'].std()
portfolio_min = portfolio_returns['portfolio_return'].min()
portfolio_max = portfolio_returns['portfolio_return'].max()

# Compute cumulative returns of portfolio and S&P 500
portfolio_cum_return = (1 + portfolio_returns['portfolio_return']).cumprod() - 1
SP500_cum_return = (1 + df_SP500_selected['Change %']).cumprod() - 1

# Plot cumulative returns of portfolio and S&P 500
plt.plot(portfolio_returns['date'], portfolio_cum_return, label='RH portfolio')
plt.plot(df_SP500.index, SP500_cum_return, label='S&P 500')
plt.legend()
plt.xlabel('Date')
plt.ylabel('Cumulative return')

# Compute Sharpe ratio of portfolio and S&P 500
risk_free_rate = 0.02
portfolio_sharpe = (portfolio_mean - risk_free_rate) / portfolio_std
SP500_sharpe = (df_SP500['SP500_return'].mean() - risk_free_rate) / df_SP500['SP500_return'].std()

# Print basic statistics and Sharpe ratios of portfolio and S&P 500
print('RH portfolio statistics:')
print('Mean: {:.4f}'.format(portfolio_mean))
print('Standard deviation: {:.4f}'.format(portfolio_std))
print('Minimum: {:.4f}'.format(portfolio_min))
print('Maximum: {:.4f}'.format(portfolio_max))
print('Sharpe ratio: {:.4f}'.format(portfolio_sharpe))
print()
print('S&P 500 statistics:')
print('Mean: {:.4f}'.format(df_SP500['SP500_return'].mean()))
print('Standard deviation: {:.4f}'.format(df_SP500['SP500_return'].std()))
print('Minimum: {:.4f}'.format(df_SP500['SP500_return'].min()))
print('Maximum: {:.4f}'.format(df_SP500['SP500_return'].max()))
print('Sharpe ratio: {:.4f}'.format(SP500_sharpe))


KeyError: 'portfolio_return'

### QUESTION 2 :
En utilisant le portefeuille que vous avez construit dans la question précédente, calculez les
mesures de risque suivantes :